### sử dụng với dữ liệu creditcard https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud?resource=download

In [1]:
from sklearn.ensemble import RandomForestClassifier
from collections import deque
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('creditcard.csv')
def create_datastream(data, batch_size):
    datastream = []
    for i in range(0, len(data), batch_size):
        batch = data.iloc[i:i+batch_size]
        datastream.append(batch)
    return datastream
batch_size = 100
S = create_datastream(df, batch_size)

In [3]:
def train_random_forest_classifier(X, y, n_estimators=10):
    """
    Huấn luyện bộ phân loại Random forest trên dữ liệu X, y.
    n_estimators: Số lượng cây trong Random forest.
    """
    rf = RandomForestClassifier(n_estimators=n_estimators)
    rf.fit(X, y)
    return rf

In [4]:
def adaptive_ensemble_size(C, sample, unique_label, anpha, min_num = 3):
    """
    Cơ chế adaptive_esemble 
    C : Bộ phân loại yếu
    sample
    unique_label : Tập hợp nhãn trong luồng dữ liệu
    anpha : hyper parameter
    min_num : số lượng C tối thiểu 
    """
    probability_list = []
    for i in range(len(unique_label)):
        li = [] 
        for j in range(len(C)):
            probabilities = C[0].predict_proba(sample)[0]
            probability_dict = {label: prob for label, prob in zip(C[0].classes_, probabilities)}
            latest_proba = probability_dict.get(unique_label[i], 0)
            probabilities1 = C[j].predict_proba(sample)[0]
            probability_dict1 = {label: prob for label, prob in zip(C[j].classes_, probabilities1)}
            current_proba = probability_dict1.get(unique_label[i], 0)
            if len(li) < min_num:
                li.append(current_proba)
            else:
                if abs(current_proba - latest_proba) < anpha :
                    li.append(current_proba)
                else: 
                    break   
        probability_list.append(li)     
    return probability_list

In [5]:
def linear_regression(x, y):
    """Tính hồi quy tuyến tính"""
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    return m, c

def tendency_prediction(probability_list, Y, epsilon=0.01):
    """
    Thực hiện cơ chế dự đoán xu hướng
    Input:
    - probability_list: Danh sách xác suất cho mỗi mẫu
    - Y: Danh sách các lớp trong luồng dữ liệu
    - epsilon: Hệ số trọng số (mặc định là 0.01)
    Output:
    - Ps: Lớp dự đoán cho mẫu
    """
    predicted_probabilities = []
    for i in range(len(Y)):
        li = probability_list[i]
        x = np.arange(1, len(li) + 1)
        y = np.array(li)
        slope, intercept = linear_regression(x, y)
        next_value = slope * (len(li) + 1) + intercept
        li.append(next_value)
        weighted_prob = sum([li[x] * (1 + x * epsilon) for x in range(len(li))]) / len(li)
        predicted_probabilities.append(weighted_prob)
    Ps = Y[np.argmax(predicted_probabilities)]
    
    return Ps


In [6]:

def process_data_stream(S, m, k, unique_labels):
    """
    Processes data stream S using a weak Random Forest classifier.
    - S: Data stream (list of data blocks Bi)
    - m: Max size of classifier set C
    - k: Min size of C to make predictions
    """
    C = deque(maxlen=m) 
    pre = []
    for Bi in S:
        block_predictions = []
        X = Bi.iloc[:, :-1] 
        y = Bi.iloc[:, -1]   
        Ci = train_random_forest_classifier(X, y)
        C.append(Ci)
        while len(C) < k :
            C.append(Ci)
        for index, row in Bi.iterrows():
            sample = pd.DataFrame([row[:-1]], columns=Bi.columns[:-1])
            anpha = (1500 / len(Bi)) * 0.2
            selected_classifiers = adaptive_ensemble_size(C, sample, unique_labels, anpha)
            pre_sample = tendency_prediction(selected_classifiers, unique_labels)
            block_predictions.append(pre_sample)
        pre.append(block_predictions)
    return C, pre
m = 15
k = 3
unique_labels = set(df.iloc[:, -1])
unique_labels = list(unique_labels)
C, pre = process_data_stream(S, m, k, unique_labels)


KeyboardInterrupt: 